In [1]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'bone-fracture-detection-computer-vision-project:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F4296838%2F7391382%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240512%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240512T191317Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D7b342eb3e08a3b69c715e18c9af1ee33aa4b61a079f7b974d351fc30865d0fe71669a7944a9a75fb0856b074075b7ef3e3a2d53ad1ccbcd877828e77b2cac0c46b27228c3671a7b83e4f83693e8f4abf8ea67be8ad01342452f3e4dec232d7f47b82f2ed3385b001e2846bd50c483eca045b2b2efeef9a3b5c6d1eed4fef5da7f53119000d1926a3d8fcb8bc05e791d967fe292fb7c8ea99a0a6684a03174a189a9adf60ba98a4552aa665de914fc29526eab9055c066a7ecb8399aac62c10efae69eba053f393935438f38bce15ef60125f417674e599eac293411849b4c0783ecf0d4ef86ecc9be891b86454eb3d809702a5bfff4c37a5e06477ff5b631416'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


[==================================================] 43644754 bytes downloaded
Downloaded and uncompressed: bone-fracture-detection-computer-vision-project
Data source import complete.


In [2]:
import numpy as np
import pandas as pd
import PIL
import PIL.Image
import matplotlib.pyplot as plt
!pip install GPUtil
import torch
from GPUtil import showUtilization as gpu_usage
from numba import cuda
import glob

  Preparing metadata (setup.py) ... done
  Created wheel for GPUtil: filename=GPUtil-1.4.0-py3-none-any.whl size=7394 sha256=0a2946b3e3e9ca0ebd5c1dbbcee86f9f85621110b02764f122fea5bdbee7908a
  Stored in directory: /root/.cache/pip/wheels/a9/8a/bd/81082387151853ab8b6b3ef33426e98f5cbfebc3c397a9d4d0
Successfully built GPUtil


In [3]:
%pip install ultralytics
import ultralytics
from ultralytics import YOLO
ultralytics.checks()

Ultralytics YOLOv8.2.14 🚀 Python-3.10.12 torch-2.2.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
Setup complete ✅ (2 CPUs, 12.7 GB RAM, 29.9/78.2 GB disk)


In [4]:
test_loc='/kaggle/input/bone-fracture-detection-computer-vision-project/test/'
train_loc='/kaggle/input/bone-fracture-detection-computer-vision-project/train/'
val_loc='/kaggle/input/bone-fracture-detection-computer-vision-project/valid/'
data_loc = '/kaggle/input/bone-fracture-detection-computer-vision-project/data.yaml'

In [12]:
model = YOLO('/kaggle/working/yolov8m.pt')
model.train(data=data_loc, epochs=30, imgsz=256, batch=10)

Ultralytics YOLOv8.2.14 🚀 Python-3.10.12 torch-2.2.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=/kaggle/working/yolov8m.pt, data=/kaggle/input/bone-fracture-detection-computer-vision-project/data.yaml, epochs=30, time=None, patience=100, batch=10, imgsz=256, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train3, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=F

train: Scanning /kaggle/input/bone-fracture-detection-computer-vision-project/train/labels.cache... 3631 images, 1827 backgrounds, 0 corrupt: 100%|██████████| 3631/3631 [00:00<?, ?it/s]

albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01), CLAHE(p=0.01, clip_limit=(1, 4.0), tile_grid_size=(8, 8))



/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
val: Scanning /kaggle/input/bone-fracture-detection-computer-vision-project/valid/labels.cache... 348 images, 175 backgrounds, 0 corrupt: 100%|██████████| 348/348 [00:00<?, ?it/s]


Plotting labels to runs/detect/train3/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.000909, momentum=0.9) with parameter groups 77 weight(decay=0.0), 84 weight(decay=0.00046875), 83 bias(decay=0.0)
TensorBoard: model graph visualization added ✅
Image sizes 256 train, 256 val
Using 2 dataloader workers
Logging results to runs/detect/train3
Starting training for 30 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

      30/30      1.92G      1.771      1.421      1.276          2        256: 100%|██████████| 364/364 [00:59<00:00,  6.12it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 18/18 [00:01<00:00,  9.33it/s]


                   all        348        204      0.331      0.231      0.219     0.0785

30 epochs completed in 0.553 hours.
Optimizer stripped from runs/detect/train3/weights/last.pt, 52.0MB
Optimizer stripped from runs/detect/train3/weights/best.pt, 52.0MB

Validating runs/detect/train3/weights/best.pt...
Ultralytics YOLOv8.2.14 🚀 Python-3.10.12 torch-2.2.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 218 layers, 25843813 parameters, 0 gradients, 78.7 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 18/18 [00:04<00:00,  4.04it/s]


                   all        348        204      0.405      0.248      0.217     0.0842
        elbow positive        348         29     0.0454     0.0345     0.0265    0.00753
      fingers positive        348         48      0.281      0.125      0.138     0.0338
      forearm fracture        348         43      0.531      0.349      0.368      0.153
               humerus        348         36      0.738      0.472        0.5      0.214
     shoulder fracture        348         20      0.307      0.399        0.2     0.0788
        wrist positive        348         28      0.526      0.107     0.0702     0.0177
Speed: 0.1ms preprocess, 4.2ms inference, 0.0ms loss, 2.7ms postprocess per image
Results saved to runs/detect/train3


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1, 2, 4, 5, 6])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7ef6561b38b0>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
     

In [13]:
model_test = YOLO('runs/detect/train3/weights/best.pt')

In [16]:
test_image = glob.glob("/kaggle/input/bone-fracture-detection-computer-vision-project/test/images/*.jpg")
out = model_test(test_image)


0: 256x256 (no detections), 3.2ms
1: 256x256 (no detections), 3.2ms
2: 256x256 (no detections), 3.2ms
3: 256x256 (no detections), 3.2ms
4: 256x256 (no detections), 3.2ms
5: 256x256 (no detections), 3.2ms
6: 256x256 (no detections), 3.2ms
7: 256x256 (no detections), 3.2ms
8: 256x256 (no detections), 3.2ms
9: 256x256 (no detections), 3.2ms
10: 256x256 (no detections), 3.2ms
11: 256x256 (no detections), 3.2ms
12: 256x256 (no detections), 3.2ms
13: 256x256 1 shoulder fracture, 3.2ms
14: 256x256 1 shoulder fracture, 3.2ms
15: 256x256 (no detections), 3.2ms
16: 256x256 (no detections), 3.2ms
17: 256x256 (no detections), 3.2ms
18: 256x256 (no detections), 3.2ms
19: 256x256 (no detections), 3.2ms
20: 256x256 (no detections), 3.2ms
21: 256x256 (no detections), 3.2ms
22: 256x256 (no detections), 3.2ms
23: 256x256 1 fingers positive, 1 wrist positive, 3.2ms
24: 256x256 (no detections), 3.2ms
25: 256x256 (no detections), 3.2ms
26: 256x256 (no detections), 3.2ms
27: 256x256 (no detections), 3.2ms
